# **_Bot Library_**

**_Collection of trading bots and various strategies._**

## **_Libraries_**

In [2]:
# Keys
from __Alpaca_API_Keys__ import *

# Trading
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest
from alpaca.trading.enums import OrderSide, OrderType, TimeInForce

# Time 
import datetime
import threading
import time
import pytz

# Data
import numpy as np
import pandas as pd
import yfinance as yf

# Web-Scraping
from bs4 import BeautifulSoup as bs
import requests

# Recording
import plotly.graph_objects as go
import logging

## **_Trading Bots and Threader_**

In [9]:
class BBands:
    """
    ## Bollinger Bands

    ### Description:
    This class allows the user to create multiple instances of "bots" with
    varying parameters which are based on a BBands strategy.

    You can also inspect the trades made by a bot by using the plotting 
    method which gives a detailed view of all the trades in relation to the
    price and bands.

    ### Strategy Logic:
    When price is below the lower band, a buy order is made.

    When price is above the upper band, a sell order is made.

    ### Args:
        - client (alpaca trading client): Trading client linking to your Alpaca account
        - symbol (string): The stock symbol you wish to trade on
        - moving_average (int): The moving average window
        - standard_deviation (float): The coefficient for the standard deviation
        - quantity (int): The number of stocks you want to buy or sell at a time
        - timeframe (int): Your chosen trading timeframe in seconds

    ### Methods:
        - Features: Prints out all attributes and details about your trading object
        - Bot: Runs a BBands strategy based on given parameters
        - Plotting: Returns a plot of all trades made while your object was running
    """
    
    def __init__(self, client, symbol, moving_average, standard_deviation, quantity, timeframe):
        self.cl = client
        self.sb = symbol
        self.ma = moving_average
        self.sd = standard_deviation
        self.qt = quantity
        self.tf = timeframe
        print(f"{self.sb} BBands Bot")
        print(f"MA: {self.ma}, SD: {self.sd}")
        print("Initialised")
        print("")

    def Features(self):
        """
        ## Features

        ### Description:
        A method to print all current attributes and details about the BBands bot
        you've created. 

        ### Args:
        - self (instance): Instance of the class
        """

        # Default features
        print(f"Client:                  {self.cl}")
        print(f"Symbol:                  {self.sb}")
        print(f"Quantity:                {self.qt}")
        print(f"Timeframe:               {self.tf}")

        # Strategy dependant features
        print(f"Moving Average:          {self.ma}")
        print(f"Standard Deviation:      {self.sd}")

        # Time dependant features
        try:
            print(f"Trade count:             {len(self.tr)}")
            print(f"Beginning of Bot:        {len(self.bg)}")
            print(f"Revolution count:        {len(self.cb)}")
            print(f"Active revolution count: {len(self.cb.loc[self.bg:])}")
        except Exception as e:
            pass

    def Bot(self, stop_time, cancel, logger):
        """
        ## Bot

        ### Description:
        A method to run the strategy which places trades using the Alpaca API.
        First, close data is webscraped from Yahoo Finance. It's then ordered
        into a dataframe. The BBands strategy is then ran on the data. The 
        trades are then recorded in the History function.

        ### Args: 
        - self (instance): Instance of the class
        - stop_time (string): A structured datetime string in the format "Y-m-d H:M:S"
        - cancel (bool): A Bool which determines if all positions should be canceled once the bot finishes
        - logger (module): Logger object to monitor script status
        """

        # Set start to today's market open
        current = datetime.datetime.now()
        beginning = f"{current.year}-{current.month}-{current.day} {current.hour}:{current.minute}:{current.second}"
        beginning = datetime.datetime.strptime(beginning, "%Y-%m-%d %H:%M:%S")

        # Setting timezone to UTC
        timezone = pytz.timezone("Europe/Dublin")
        beginning = timezone.localize(beginning)
        self.bg = beginning

        # Set the stop time
        stop_time = datetime.datetime.strptime(stop_time, "%Y-%m-%d %H:%M:%S")
        stop_time = timezone.localize(stop_time)
        self.st = stop_time

        # Other attributes
        self.cp = cancel
        self.lg = logger

        # Functions
        def Sleep(self):
            """
            ## Sleep

            ### Description:
            A function that calculates number of seconds until the next minute.

            ### Args:
            - self (instance): Instance of the class

            ### Returns:
            - sleep (float): Number of seconds until next minute
            """

            # Calculating seconds until next interval
            current = time.time()
            sleep = self.tf - (current % self.tf)
            return sleep
        
        def Webscraping(self):
            """
            ## Webscraping

            ### Description:
            A function to webscrape real-time close data from the Yahoo Finance website.

            ### Args:
            - self (instance): Instance of the class

            ### Returns:
            - price (float): Real-time close price
            """

            # Initialising vars to access Yahoo Finance
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{self.sb}"

            # Extracting HTML code
            request = requests.get(url, headers=headers)
            self.lg.info(f"{self.sb}: WS: Request made")
            htmlcode = bs(request.text, "html.parser")
            self.lg.info(f"{self.sb}: WS: Code gathered")

            # Parsing the code to find the current price
            price = float(htmlcode.find("fin-streamer", {"class": "livePrice svelte-mgkamr"}).text.replace(',', ''))
            self.lg.info(f"{self.sb}: WS: Price created {price}")

            return price

        def Data(self, price, previous):
            """
            ## Data

            ### Description:
            A function that takes the previous webscraped data and most recent addition and combines
            them. The data is then used to create features and parameters like the upper and lower 
            bands which give signals later on.

            ### Args:
            - self (instance): Instance of the class
            - price (float): Real-time close price
            - previous (df): All price data and specific features accumulated while running
                
            ### Returns:
            - combined (df): "previous" df combined with new price data. Becomes "previous" df in next iteration
            """
            
            # Getting current time info for the dataframe index
            current = datetime.datetime.now(pytz.timezone("Europe/Dublin"))
            self.lg.info(f"{self.sb}: DT: Date created {current}")
            
            try:
                # Assembling the price and feature dataframe
                dataframe = pd.DataFrame({"Price": price}, index=[current])
                combined = pd.concat([previous, dataframe])
                self.lg.info(f"{self.sb}: DT: Combined created")
            except Exception as e:
                print(f"Data ERROR: {e}")
                return previous

            # Creating feature columns for the chosen strategy
            combined["Moving Average"] = combined["Price"].rolling(self.ma).mean()
            combined["Standard Deviation"] = combined["Price"].rolling(self.ma).std()
            combined["Upper Band"] = combined["Moving Average"] + (self.sd * combined["Standard Deviation"])
            combined["Lower Band"] = combined["Moving Average"] - (self.sd * combined["Standard Deviation"])

            return combined
            
        def Orders(self, side):
            """
            ## Orders

            ### Description:
            A function to send orders to the Alpaca website.

            ### Args:
            - self (instance): Instance of the class
            - side (OrderSide): order to buy or sell
            """

            # Order bracket
            order_data = MarketOrderRequest(
                symbol=self.sb,
                qty=self.qt,
                side=side,
                type=OrderType.MARKET,
                time_in_force=TimeInForce.GTC)
            self.lg.info(f"{self.sb}: OD: Order Bracket created")

            # Submitting the order bracket
            self.cl.submit_order(order_data=order_data)
            self.lg.info(f"{self.sb}: OD: Order submitted")

        def Strategy(self, combined):
            """
            ## Strategy

            ### Description:
            A function to check whether the strategy's conditions have been met. If so,
            an order is sent using the "Orders" function.

            ### Args:
            - self (instance): Instance of the class
            - combined (df): Df of all strategy features

            ### Returns:
            - None
            """

            # Condition 1: If the close price is above the upperband, Sell
            if combined["Price"].iloc[-1] > combined["Upper Band"].iloc[-1]:
                Orders(self, OrderSide.SELL)
                print("Sold")
                self.lg.info(f"{self.sb}: ST: Sell")
            
            # Condition 2: If the close price is below the lowerband, Buy
            elif combined["Price"].iloc[-1] < combined["Lower Band"].iloc[-1]:
                Orders(self, OrderSide.BUY)
                print("Bought")
                self.lg.info(f"{self.sb}: ST: Buy")

        def History(self, previous):
            """
            ## History

            ### Description:
            A function to gather and record trades made while running

            ### Args:
            - self (instance): Instance of the class
            - previous (df): All trades accumulated and recorded while running

            ### Returns:
            - trades (df): Combined dataframe of the most recent trade and all previous trades recorded while running 
            """

            # Getting 50 most recent orders
            orders = self.cl.get_orders(filter=GetOrdersRequest(status="closed", symbols=[self.sb]))
            self.lg.info(f"{self.sb}: HT: Order data gathered")
            self.lg.info(f"{self.sb}: HT: Orders length: {len(orders)}")

            # Extracting specifics from most recent trade
            if len(orders) > 0:
                fill = orders[0].filled_avg_price
                time = orders[0].filled_at
                side = orders[0].side
            else:
                fill = None
                time = None
                side = None

            # Converting time to UTC
            if time is not None:
                time = time.astimezone(pytz.timezone("Europe/Dublin"))

            # Converting the fill price from a string to a float
            if fill is not None:
                self.lg.info(f"{self.sb}: HT: Fill is a number: {fill}")
                fill = float(fill)

            # Creating data for plotting based on the trade
            if side == OrderSide.BUY:
                colour = "green"
                marker = "triangle-up"
            elif side == OrderSide.SELL:
                colour = "red"
                marker = "triangle-down"
            else:
                colour = None
                marker = None

            # Creating a dataframe to store trade data
            trade = pd.DataFrame(dict({"Avg Fill Price":fill, "Side":side, "Colour":colour, "Marker":marker}), index=[time])
            self.lg.info(f"{self.sb}: HT: Trade dataframe created")

            # Combining the previous trades and most recent trade
            trades = pd.concat([previous, trade])
            self.lg.info(f"{self.sb}: HT: Trade dataframe combined")

            # Removing duplicate if present
            trades = trades.drop_duplicates(keep="last")
            self.lg.info(f"{self.sb}: HT: Duplicates removed")

            return trades

        # Placeholders
        combined = pd.DataFrame()
        trades = pd.DataFrame()

        self.lg.info(f"{self.sb}: Bot Initialised")

        # trading loop until stoptime
        while datetime.datetime.now(pytz.timezone("Europe/Dublin")) < stop_time:

            # Sleeping until next interval in chosen timeframe
            time.sleep(Sleep(self))
            
            # Webscraping close data from the Yahoo Finance website
            try:
                price = Webscraping(self)
            except Exception as e:
                self.lg.error(f"{self.sb}: Webscraping ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Webscraping ERROR", e)

            # Creating features for chosen strategy
            try:
                combined = Data(self, price, combined)
            except Exception as e:
                self.lg.error(f"{self.sb}: Data ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Data ERROR:", e)
        
            # Submitting orders if conditions are met on calculated features
            try:
                Strategy(self, combined)
            except Exception as e:
                self.lg.error(f"{self.sb}: Strategy ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Strategy ERROR:", e)

            # Recording trades if any were made
            try:
                trades = History(self, trades)
            except Exception as e:
                self.lg.error(f"{self.sb}: History ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: History ERROR:", e)

        # Updating attributes with most recent data
        self.cb = combined
        self.tr = trades

        # Closing positions if True
        if self.cp == True:
            self.cl.close_all_positions(cancel_orders=True)

    def Plotting(self, interval="1m"):
        """
        ## Plotting

        ### Description:
        A method to display the trades made throughout the day and BBands.

        ### Args:
        - self (instance): Instance of the class
        - interval (string): OHLC timeframe can only be the following:
            [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

        ### Returns:
        - fig (plotly figure): The created figure of trades, indicators etc.
        """

        # Gather OHLC data first
        ohlc = yf.Ticker(self.sb)
        ohlc = ohlc.history(period="5d", interval=interval)
        ohlc.index = ohlc.index.tz_convert("Europe/Dublin")

        # Check trades
        trades = self.tr[self.tr.index.notna()]

        # Setting the market open time
        timezone = pytz.timezone("UTC")
        current = datetime.datetime.now()
        mkop = f"{current.year}-{current.month}-{current.day} 14:30:00"
        mkop = datetime.datetime.strptime(mkop, "%Y-%m-%d %H:%M:%S")
        mkop = timezone.localize(mkop)

        # Clearing irrelevant data based on the market open
        if self.bg < mkop:
            trades = trades.loc[mkop:]
            indicators = self.cb.loc[mkop:]
            ohlc = ohlc.loc[mkop:]

        # Clearing irrelevant data based on the bot beginning
        elif self.bg > mkop:
            trades = trades.loc[self.bg:]
            indicators = self.cb.loc[self.bg:]
            ohlc = ohlc.loc[self.bg:]

        # Creating a figure
        fig = go.Figure()

        # Plotting the trades using a scatter
        fig.add_trace(go.Scatter(x=trades.index,
                                 y=trades["Avg Fill Price"],
                                 mode="markers",
                                 name="Buys + Sells",
                                 marker=dict(color=trades["Colour"], size=20, symbol=trades["Marker"])))
        
        # Plotting the close prices we collected through webscraping
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Price"],
                                 mode="lines+markers",
                                 name="Price"))
        
        # Plotting the Moving Average using a line plot
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Moving Average"],
                                 mode="lines",
                                 name="Moving Average"))
        
        # Plotting the Upper Band using a line plot
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Upper Band"],
                                 mode="lines",
                                 name="Upper Band"))
        
        # Plotting the Lower Band using a line plot
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Lower Band"],
                                 mode="lines",
                                 name="Lower Band"))
        
        # Plotting the OHLC data using candlesticks
        fig.add_trace(go.Candlestick(x=ohlc.index,
                                     open=ohlc["Open"],
                                     high=ohlc["High"],
                                     low=ohlc["Low"],
                                     close=ohlc["Close"],
                                     name="OHLC"))
        
        # Updating the layout of the plot
        fig.update_layout(xaxis_rangeslider_visible=False, height=700,
                          paper_bgcolor="rgba(70,70,70,1)",
                          plot_bgcolor="rgba(230,230,230,1)",
                          xaxis=dict(title="Date & Time",
                                     tickfont=dict(color="white"),
                                     titlefont=dict(color="white"),
                                     gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                          yaxis=dict(title="Price",
                                     tickfont=dict(color="rgba(230,230,230,1)"),
                                     titlefont=dict(color="rgba(230,230,230,1)"),
                                     gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                          shapes = [go.layout.Shape(type='rect',
                                                    xref='paper',
                                                    yref='paper',
                                                    x0=0,
                                                    y0=0,
                                                    x1=1,
                                                    y1=1,
                                                    line={'width': 3, 'color': 'black'})],
                          legend=dict(font=dict(color="rgba(230,230,230,1)")))
                             

        # Displaying and returning the final figure
        fig.show()
        return fig

In [11]:
class MA_Cross:
    """
    ## Moving Average Crossover

    ### Description:
    This class allows the user to create multiple instances of "bots" with
    varying parameters which are based on an moving average crossover strategy.

    You can also inspect the trades made by a bot by using the plotting 
    method which gives a detailed view of all the trades in relation to the
    price and different moving averages.

    ### Strategy Logic:
    when the fast moving average crosses above the slow moving average, a buy order is made.

    When the fast moving average crosses below the slow moving average, a sell order is made.

    ### Args:
    - client (Alpaca trading client): Trading client linking to your Alpaca account
    - symbol (String): The stock symbol you wish to trade on
    - fast_window (Int): The fast moving average window
    - slow_window (Int): The slow moving average window
    - quantity (Int): The number of stocks you want to buy or sell at a time
    - timeframe (Int): Your chosen trading timeframe in seconds

    ### Methods:
    - Features: Prints out all attributes and details about your trading object
    - Bot: Runs an MA Crossover strategy based on given parameters
    - Plotting: Returns a plot of all trades made while your object was running
    """

    def __init__(self, client, symbol, fast_window, slow_window, quantity, timeframe):
        self.cl = client
        self.sb = symbol
        self.fw = fast_window
        self.sw = slow_window
        self.qt = quantity
        self.tf = timeframe
        print(f"{self.sb} MA Cross Bot")
        print(f"Fast MA: {self.fw}, Slow MA: {self.sw}")
        print("Initialised")
        print("")

    def Features(self):
        """
        ## Features

        ### Description:
        A method to print all current attributes and details about the BBands bot
        you've created. 

        ### Args:
        - self (Instance): Instance of the class
        """

        # Default features
        print(f"Client:                  {self.cl}")
        print(f"Symbol:                  {self.sb}")
        print(f"Quantity:                {self.qt}")
        print(f"Timeframe:               {self.tf}")

        # Strategy dependant features
        print(f"Fast Window:             {self.fw}")
        print(f"Slow Window:             {self.sw}")

        # Time dependant features
        try:
            print(f"Trade count:             {len(self.tr)}")
            print(f"Beginning of Bot:        {len(self.bg)}")
            print(f"Revolution count:        {len(self.cb)}")
            print(f"Active revolution count: {len(self.cb.loc[self.bg:])}")
        except Exception as e:
            pass

    def Bot(self, stop_time, cancel, logger):
        """
        ## Bot

        ### Description:
        A method to run the MA Crossover strategy which places trades using the Alpaca API.
        First, close data is webscraped from Yahoo Finance. It's then ordered
        into a dataframe. The MA Crossover strategy is then ran on the data. The 
        trades are then recorded in the History function.

        ### Args: 
        - self (Instance): Instance of the class
        - stop_time (String): A structured datetime string in the format "Y-m-d H:M:S"
        - cancel (Bool): A Bool which determines if all positions should be canceled once the bot finishes
        - logger (Module): Logger object to monitor script status
        """

        # Set start to today's market open
        current = datetime.datetime.now()
        beginning = f"{current.year}-{current.month}-{current.day} {current.hour}:{current.minute}:{current.second}"
        beginning = datetime.datetime.strptime(beginning, "%Y-%m-%d %H:%M:%S")

        # Setting timezone to UTC
        timezone = pytz.timezone("Europe/Dublin")
        beginning = timezone.localize(beginning)
        self.bg = beginning

        # Set the stop time
        stop_time = datetime.datetime.strptime(stop_time, "%Y-%m-%d %H:%M:%S")
        stop_time = timezone.localize(stop_time)
        self.st = stop_time

        # Other attributes
        self.cp = cancel
        self.lg = logger

        # Functions
        def Sleep(self):
            """
            ## Sleep

            ### Description:
            A function that calculates number of seconds until the next minute.

            ### Args:
            - self (Instance): Instance of the class

            ### Returns:
            - sleep (Float): Number of seconds until next minute
            """

            # Calculating seconds until next interval
            current = time.time()
            sleep = self.tf - (current % self.tf)
            return sleep
        
        def Webscraping(self):
            """
            ## Webscraping

            ### Description:
            A function to webscrape real-time close data from the Yahoo Finance website.

            ### Args:
            - self (Instance): Instance of the class

            ### Returns:
            - price (Float): Real-time close price
            """

            # Initialising vars to access Yahoo Finance
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"}
            url = f"https://finance.yahoo.com/quote/{self.sb}"

            # Extracting HTML code
            request = requests.get(url, headers=headers)
            self.lg.info(f"{self.sb}: WS: Request made")
            htmlcode = bs(request.text, "html.parser")
            self.lg.info(f"{self.sb}: WS: Code gathered")

            # Parsing the code to find the current price
            price = float(htmlcode.find("fin-streamer", {"class": "livePrice svelte-mgkamr"}).text.replace(',', ''))
            self.lg.info(f"{self.sb}: WS: Price created {price}")

            return price

        def Data(self, price, previous):
            """
            ## Data

            ### Description:
            A function that takes the previous webscraped data and most recent addition and combines
            them. The data is then used to create features and parameters like the fast and slow 
            moving averages which give signals later on.

            ### Args:
            - self (Instance): Instance of the class
            - price (Float): Real-time close price
            - previous (Df): All price data and specific features accumulated while running
                
            ### Returns:
            - combined (Df): "previous" df combined with new price data. Becomes "previous" df in next iteration
            """
            
            # Getting current time info for the dataframe index
            current = datetime.datetime.now(pytz.timezone("Europe/Dublin"))
            self.lg.info(f"{self.sb}: DT: Date created {current}")
            
            try:
                # Assembling the price and feature dataframe
                dataframe = pd.DataFrame({"Price": price}, index=[current])
                combined = pd.concat([previous, dataframe])
                self.lg.info(f"{self.sb}: DT: Combined created")
            except Exception as e:
                print(f"Data ERROR: {e}")
                return previous

            # Creating feature columns for the chosen strategy
            combined["Fast MA"] = combined["Price"].rolling(self.fw).mean()
            combined["Slow MA"] = combined["Price"].rolling(self.sw).mean()

            return combined
            
        def Orders(self, side):
            """
            ## Orders

            ### Description:
            A function to send orders to the Alpaca website.

            ### Args:
            - self (instance): Instance of the class
            - side (OrderSide): order to buy or sell
            """

            # Order bracket
            order_data = MarketOrderRequest(
                symbol=self.sb,
                qty=self.qt,
                side=side,
                type=OrderType.MARKET,
                time_in_force=TimeInForce.GTC)
            self.lg.info(f"{self.sb}: OD: Order Bracket created")

            # Submitting the order bracket
            self.cl.submit_order(order_data=order_data)
            self.lg.info(f"{self.sb}: OD: Order submitted")

        def Strategy(self, combined):
            """
            ## Strategy

            ### Description:
            A function to check whether the strategy's conditions have been met. If so,
            an order is sent using the "Orders" function.

            ### Args:
            - self (instance): Instance of the class
            - combined (df): Df of all strategy features
            """

            # Condition 1: If the close price is above the upperband, Sell
            if (combined["Fast MA"].iloc[-2] < combined["Slow MA"].iloc[-2]) and (combined["Fast MA"].iloc[-1] > combined["Slow MA"].iloc[-1]):
                Orders(self, OrderSide.BUY)
                print("Bought")
                self.lg.info(f"{self.sb}: ST: Buy")
            
            # Condition 2: If the close price is below the lowerband, Buy
            elif (combined["Fast MA"].iloc[-2] > combined["Slow MA"].iloc[-2]) and (combined["Fast MA"].iloc[-1] < combined["Slow MA"].iloc[-1]):
                Orders(self, OrderSide.SELL)
                print("Sold")
                self.lg.info(f"{self.sb}: ST: Sell")

        def History(self, previous):
            """
            ## History

            ### Description:
            A function to gather and record trades made while running

            ### Args:
            - self (instance): Instance of the class
            - previous (df): All trades accumulated and recorded while running

            ### Returns:
            - trades (df): Combined dataframe of the most recent trade and all previous trades recorded while running 
            """

            # Getting 50 most recent orders
            orders = self.cl.get_orders(filter=GetOrdersRequest(status="closed", symbols=[self.sb]))
            self.lg.info(f"{self.sb}: HT: Order data gathered")
            self.lg.info(f"{self.sb}: HT: Orders length: {len(orders)}")

            # Extracting specifics from most recent trade
            if len(orders) > 0:
                fill = orders[0].filled_avg_price
                time = orders[0].filled_at
                side = orders[0].side
            else:
                fill = None
                time = None
                side = None

            # Converting time to UTC
            if time is not None:
                time = time.astimezone(pytz.timezone("Europe/Dublin"))

            # Converting the fill price from a string to a float
            if fill is not None:
                self.lg.info(f"{self.sb}: HT: Fill is a number: {fill}")
                fill = float(fill)

            # Creating data for plotting based on the trade
            if side == OrderSide.BUY:
                colour = "green"
                marker = "triangle-up"
            elif side == OrderSide.SELL:
                colour = "red"
                marker = "triangle-down"
            else:
                colour = None
                marker = None

            # Creating a dataframe to store trade data
            trade = pd.DataFrame(dict({"Avg Fill Price":fill, "Side":side, "Colour":colour, "Marker":marker}), index=[time])
            self.lg.info(f"{self.sb}: HT: Trade dataframe created")

            # Combining the previous trades and most recent trade
            trades = pd.concat([previous, trade])
            self.lg.info(f"{self.sb}: HT: Trade dataframe combined")

            # Removing duplicate if present
            trades = trades.drop_duplicates(keep="last")
            self.lg.info(f"{self.sb}: HT: Duplicates removed")

            return trades

        # Placeholders
        combined = pd.DataFrame()
        trades = pd.DataFrame()

        self.lg.info(f"{self.sb}: Bot Initialised")

        # trading loop until stoptime
        while datetime.datetime.now(pytz.timezone("Europe/Dublin")) < stop_time:

            # Sleeping until next interval in chosen timeframe
            time.sleep(Sleep(self))
            
            # Webscraping close data from the Yahoo Finance website
            try:
                price = Webscraping(self)
            except Exception as e:
                self.lg.error(f"{self.sb}: Webscraping ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Webscraping ERROR", e)

            # Creating features for chosen strategy
            try:
                combined = Data(self, price, combined)
            except Exception as e:
                self.lg.error(f"{self.sb}: Data ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Data ERROR:", e)
        
            # Submitting orders if conditions are met on calculated features
            try:
                Strategy(self, combined)
            except Exception as e:
                self.lg.error(f"{self.sb}: Strategy ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: Strategy ERROR:", e)

            # Recording trades if any were made
            try:
                trades = History(self, trades)
            except Exception as e:
                self.lg.error(f"{self.sb}: History ERROR: {e}", exc_info=True)
                print(f"{self.sb}, {datetime.datetime.now()}: History ERROR:", e)

        # Updating attributes with most recent data
        self.cb = combined
        self.tr = trades

        # Closing positions if True
        if self.cp == True:
            self.cl.close_all_positions(cancel_orders=True)

    def Plotting(self, interval="1m"):
        """
        ## Plotting

        ### Description:
        A method to display the trades made throughout the day and moving averages.

        ### Args:
        - self (instance): Instance of the class
        - interval (string): OHLC timeframe can only be the following:
            [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]

        ### Returns:
        - fig (plotly figure): The created figure of trades, indicators etc.
        """

        # Gather OHLC data first
        ohlc = yf.Ticker(self.sb)
        ohlc = ohlc.history(period="5d", interval=interval)
        ohlc.index = ohlc.index.tz_convert("Europe/Dublin")

        # Check trades
        trades = self.tr[self.tr.index.notna()]

        # Setting the market open time
        timezone = pytz.timezone("Europe/Dublin")
        current = datetime.datetime.now()
        mkop = f"{current.year}-{current.month}-{current.day} 14:30:00"
        mkop = datetime.datetime.strptime(mkop, "%Y-%m-%d %H:%M:%S")
        mkop = timezone.localize(mkop)

        # Clearing irrelevant data based on the market open
        if self.bg < mkop:
            trades = trades.loc[mkop:]
            indicators = self.cb.loc[mkop:]
            ohlc = ohlc.loc[mkop:]

        # Clearing irrelevant data based on the bot beginning
        elif self.bg > mkop:
            trades = trades.loc[self.bg:]
            indicators = self.cb.loc[self.bg:]
            ohlc = ohlc.loc[self.bg:]

        # Creating a figure
        fig = go.Figure()

        # Plotting the trades using a scatter
        fig.add_trace(go.Scatter(x=trades.index,
                                 y=trades["Avg Fill Price"],
                                 mode="markers",
                                 name="Buys + Sells",
                                 marker=dict(color=trades["Colour"], size=20, symbol=trades["Marker"])))
        
        # Plotting the close prices we collected through webscraping
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Price"],
                                 mode="lines+markers",
                                 name="Price"))
        
        # Plotting the fast moving average using a line plot
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Fast MA"],
                                 mode="lines",
                                 name="Fast MA"))
        
        # Plotting the slow moving average using a line plot
        fig.add_trace(go.Scatter(x=indicators.index,
                                 y=indicators["Slow MA"],
                                 mode="lines",
                                 name="Slow MA"))
        
        # Plotting the OHLC data using candlesticks
        fig.add_trace(go.Candlestick(x=ohlc.index,
                                     open=ohlc["Open"],
                                     high=ohlc["High"],
                                     low=ohlc["Low"],
                                     close=ohlc["Close"],
                                     name="OHLC"))
        
        # Updating the layout of the plot
        fig.update_layout(xaxis_rangeslider_visible=False, height=700,
                          paper_bgcolor="rgba(70,70,70,1)",
                          plot_bgcolor="rgba(230,230,230,1)",
                          xaxis=dict(title="Date & Time",
                                     tickfont=dict(color="white"),
                                     titlefont=dict(color="white"),
                                     gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                          yaxis=dict(title="Price",
                                     tickfont=dict(color="rgba(230,230,230,1)"),
                                     titlefont=dict(color="rgba(230,230,230,1)"),
                                     gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                          shapes = [go.layout.Shape(type='rect',
                                                    xref='paper',
                                                    yref='paper',
                                                    x0=0,
                                                    y0=0,
                                                    x1=1,
                                                    y1=1,
                                                    line={'width': 3, 'color': 'black'})],
                          legend=dict(font=dict(color="rgba(230,230,230,1)")))
                             

        # Displaying and returning the final figure
        fig.show()
        return fig

In [12]:
def Threader(*instances, stop_time, cancel, logger):
    threads = []

    # Creating a thread for each given bot
    for instance in instances:
        thread = threading.Thread(target=instance.Bot, args=(stop_time, cancel, logger))
        threads.append(thread)
        thread.start()

    # Joining of threads
    for thread in threads:
        thread.join()